# Select localities

In [ ]:
#import packages
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
plotly.offline.init_notebook_mode()
import cufflinks as cf

file_path='all_antibiotics_mock.csv'
ab_all=pd.read_csv(file_path,low_memory=False)
all_practices=ab_all['PRACTICE NAME'].unique().tolist() #practices list
localities= ab_all.groupby('LOCALITY').mean() #stores localities means
locality_names=ab_all['LOCALITY'].unique().tolist()
practices_baseline=ab_all['BASELINE M12 2016-17']

#BASELINES
practices_baseline=ab_all['BASELINE M12 2016-17']
localities_baesline=localities['BASELINE M12 2016-17']

del ab_all['BASELINE M12 2016-17']
del localities['BASELINE M12 2016-17']

#localities time series
localities_ts=localities.transpose() 

#calculate descriptive statistics for practices time series
X = ab_all[['PRACTICE NAME', 'Apr-17', 'May-17', 'Jun-17', 'Jul-17', 'Aug-17', 'Sep-17', 'Oct-17', 'Nov-17', 'Dec-18',
       'Jan-18', 'Feb-18', 'Mar-18']]
X=X.set_index(X['PRACTICE NAME'])
del X['PRACTICE NAME']
X=X.transpose()
descriptive=X.describe()
descriptive=descriptive.transpose()
descriptive=descriptive[['max','min','mean', 'std']]
descriptive['PRACTICE NAME']=descriptive.index
ab_all_descriptive=pd.merge(ab_all,descriptive)
ab_all=ab_all_descriptive

In [ ]:
for p in locality_names: print (p)
selected_locality=input ('Select locality:') 

#prepare time series for practices in localities
localities_df=ab_all.loc[ab_all['LOCALITY']==selected_locality]
localities_df=localities_df.iloc[:, 1:14]
practices_ts=pd.DataFrame() #dataframe for practices time series
practice_ts=pd.Series() #series for practice time series(iteration)
practices=localities_df['PRACTICE NAME'].unique().tolist() #list of practices in the locality
for index1, practice in enumerate (practices): #iterate practices in locality
        practice_ts=localities_df.loc[localities_df['PRACTICE NAME']==practice] #collect all values per practice
        practices_ts[practice]=practice_ts.iloc[0,:] 
practices_ts = practices_ts.drop("PRACTICE NAME", axis=0) #drop 1st row 
practices_ts['LOCALITY_MEAN']=practices_ts.mean(axis=1)

#chart: time series with target lines
plotly.offline.iplot({
            'data':[{'x': practices_ts.index,'y': practices_ts[col],'name': col, 'mode': 'line',}
            for col in practices_ts.columns], 
            'layout': {'title': selected_locality,'xaxis': {'title': 'Period'},'yaxis': {'title': "Value"},
                       'shapes':[{'type': 'line','x0': 'Apr-17','y0': 1.161,'x1': 'Mar-18','y1': 1.161,
                                'line': {'color': 'rgb(128, 0, 128)','width': 3,'dash': 'dot'}, 
                                 }, 
                                 {'type': 'line','x0': 'Apr-17','y0': 0.965,'x1': 'Mar-18','y1': 0.965,
                                'line': {'color': 'rgb(128, 0, 128)','width': 3,'dash': 'dot'},
                                  },],}})

#chart: box plots
cf.set_config_file(offline=True, world_readable=True, theme='ggplot')
practices_ts.iplot(kind='box')

In [ ]:
from IPython.display import HTML
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')